In [1]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from efold import DataModule, create_model, Dataset
from efold.config import device
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from efold.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np



In [7]:
dm = DataModule(
    name=["RNAstralign", "archiveII", "ribo500-blast", 'bpRNA'],
    data_type=['structure'],
    force_download=False,
    batch_size=64,
    num_workers=0,
    predict_split=0,
    shuffle_valid=False,
    structure_padding_value=0,
    min_len=100,
    max_len=500,
)
dm.setup('fit')

Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            


In [8]:
for batch in dm.train_dataloader():
    print(batch.get('length'))
    break

[122, 108, 119, 123, 122, 124, 118, 122, 118, 105, 107, 116, 118, 120, 115, 120, 117, 117, 103, 118, 115, 121, 135, 120, 136, 127, 115, 140, 148, 104, 115, 102, 126, 108, 117, 118, 122, 115, 117, 115, 115, 117, 125, 114, 123, 122, 121, 118, 105, 140, 119, 118, 120, 116, 113, 147, 121, 130, 120, 113, 119, 142, 123, 140]


In [4]:
import json

names = ["RNAstralign", "archiveII", "ribo500-blast", 'bpRNA']
data = {}
for name in names:
    with open(f'./data/{name}/data.json', 'r') as f:
        temp = json.load(f)
        data.update(temp)

In [5]:
from efold.core.util import _pad, base_pairs_to_pairing_matrix
import tqdm 

def compare_data(dt, data_line, from_ds, ref):
    from_ds = from_ds.cpu().numpy().round(4)
    
    if not dt in data_line:
        assert from_ds is None or set(from_ds) == set([-1000.]), f'{dt} is None in the json and not not in the datamodule for {ref}: {from_ds}'
        return
    L = len(data_line['sequence'])
    data_line = np.array(data_line[dt])
    
    if dt == 'structure':
        data_line = base_pairs_to_pairing_matrix(data_line, L, padding=len(from_ds), pad_value=0).numpy().round(4)

    else:
        from_ds = np.array(from_ds)[:L].round(4)
        data_line = data_line.round(4)
    
    if not np.allclose(data_line, from_ds, atol=1e-6):
        assert 0, f'{dt} keys do not match for {ref}: \ndata_line:\t{data_line.tolist()} \nfrom_ds:\t{from_ds.tolist()}'
    return

def sanity_check(data:dict, dm:DataModule):
    for batch in tqdm.tqdm(dm.train_dataloader(), total=len(dm.train_dataloader())):
        for ref, dms, shape, structure in zip(batch.get('reference'), batch.get('dms'), batch.get('shape'), batch.get('structure')):
            for dt, from_ds in zip(['dms', 'shape', 'structure'], [dms, shape, structure]):
                compare_data(dt, data[ref], from_ds, ref)

sanity_check(data, dm)

  0%|          | 1/1903 [00:00<18:36,  1.70it/s]

  1%|▏         | 27/1903 [00:20<26:11,  1.19it/s]